In [1]:
import pickle
import re
import numpy as np
import json

In [2]:
import keras.models
from keras.models import model_from_json
#from scipy.misc import imread, imresize, imshow

Using TensorFlow backend.


In [3]:
token = pickle.load(open('./token.p','rb'))

In [4]:
def loadModel():
    json_file = open('./LSTM.json','r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    #Load weights into new model
    loaded_model.load_weights('./LSTM_weights.h5')
    print("Loaded model from disk")
    
    #Compile and evaluate loaded model
    loaded_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return loaded_model

In [5]:
def utf2latam(tweetText):
    return tweetText.encode('utf-8').decode('latin1')

def normalizeText(tweet):
    tweet = re.sub(r'#(S|s)arcasm|#(I|i)rony','',tweet)
    tweet = re.sub(r'#SARCASM|#IRONY','',tweet)
    tweet = re.sub(r'https?://t\.co/.(\w|\d)+','http://link', tweet) #tweet link
    tweet = re.sub(r'@.\w*','@',tweet)
    tweet = re.sub(r'#','',tweet)
    return tweet

def text2seq(tweet):
    return np.array(token.texts_to_sequences([tweet])[0]) # Return array like [1 2 3] instead of [[1 2 3]]

def getPredict(tweet_seq):
    predictions = []
    tag = []
    for i in tweet_seq:
        predictions.append(model.predict_classes(i.reshape(len(i),1)))
    for i in predictions:
        tag.append(i.reshape(1,len(i)).mean())
    return [round(x) for x in tag]


def mkCoordinate(lat, long):
    coor = []
    for i in range(0,len(lat)):
        e = []
        e.append(lat[i])
        e.append(long[i])
        coor.append(e)
    return coor

def coor2json(coordinate):
    json_data = json.dumps(coordinate)
    return '{ "type": "Point", "coordinates": '+json_data+' }'

In [6]:
task = pickle.load(open('./results.txt','rb'))

In [7]:
task.head()

,USER,TWEET,LAT,LONG
0,unatalclara,Que ironía que el tatuaje que me hice con mi e...,-99.191996,19.357102
1,alizamakeupmx,La ironía en el nombre del lugar #mames (@ Tra...,-99.259495,19.386371
2,etgarrido1976,"@Jashua98 @jrisco El Chairo, por definición, n...",-99.259495,19.386371
3,enkianlatif,Jajaja que ironía el candidato del prian @Jose...,-99.191996,19.357102
4,Raverp,#FactOfTheDay En todos los módulos de carga de...,-99.091915,19.417023


In [8]:
value = task['TWEET'].apply(normalizeText)
task.insert(loc=2,column='TWEET_SEC',value=value)
task.head()

,USER,TWEET,TWEET_SEC,LAT,LONG
0,unatalclara,Que ironía que el tatuaje que me hice con mi e...,Que ironía que el tatuaje que me hice con mi e...,-99.191996,19.357102
1,alizamakeupmx,La ironía en el nombre del lugar #mames (@ Tra...,La ironía en el nombre del lugar mames (@ Alfr...,-99.259495,19.386371
2,etgarrido1976,"@Jashua98 @jrisco El Chairo, por definición, n...","@ @ El Chairo, por definición, no entiende la ...",-99.259495,19.386371
3,enkianlatif,Jajaja que ironía el candidato del prian @Jose...,Jajaja que ironía el candidato del prian @ se...,-99.191996,19.357102
4,Raverp,#FactOfTheDay En todos los módulos de carga de...,FactOfTheDay En todos los módulos de carga de ...,-99.091915,19.417023


In [9]:
task['TWEET_SEC']=task['TWEET_SEC'].apply(text2seq)
task.head()

,USER,TWEET,TWEET_SEC,LAT,LONG
0,unatalclara,Que ironía que el tatuaje que me hice con mi e...,"[28, 3394, 1, 8, 4176, 1, 10, 748, 22, 23, 554...",-99.191996,19.357102
1,alizamakeupmx,La ironía en el nombre del lugar #mames (@ Tra...,"[59, 3394, 9, 8, 387, 27, 447, 2833, 3, 722, 1...",-99.259495,19.386371
2,etgarrido1976,"@Jashua98 @jrisco El Chairo, por definición, n...","[3, 3, 61, 16, 7, 1493, 5, 3394]",-99.259495,19.386371
3,enkianlatif,Jajaja que ironía el candidato del prian @Jose...,"[323, 1, 3394, 8, 3100, 27, 3, 14, 57, 11]",-99.191996,19.357102
4,Raverp,#FactOfTheDay En todos los módulos de carga de...,"[136, 87, 20, 12788, 2, 2822, 2, 7165, 6, 27, ...",-99.091915,19.417023


In [10]:
model = loadModel()

Loaded model from disk


In [11]:
tag = getPredict(task['TWEET_SEC'])
task.insert(loc=2,column='CLASS',value=tag)
task.head()

,USER,TWEET,CLASS,TWEET_SEC,LAT,LONG
0,unatalclara,Que ironía que el tatuaje que me hice con mi e...,1.0,"[28, 3394, 1, 8, 4176, 1, 10, 748, 22, 23, 554...",-99.191996,19.357102
1,alizamakeupmx,La ironía en el nombre del lugar #mames (@ Tra...,0.0,"[59, 3394, 9, 8, 387, 27, 447, 2833, 3, 722, 1...",-99.259495,19.386371
2,etgarrido1976,"@Jashua98 @jrisco El Chairo, por definición, n...",0.0,"[3, 3, 61, 16, 7, 1493, 5, 3394]",-99.259495,19.386371
3,enkianlatif,Jajaja que ironía el candidato del prian @Jose...,0.0,"[323, 1, 3394, 8, 3100, 27, 3, 14, 57, 11]",-99.191996,19.357102
4,Raverp,#FactOfTheDay En todos los módulos de carga de...,1.0,"[136, 87, 20, 12788, 2, 2822, 2, 7165, 6, 27, ...",-99.091915,19.417023


In [12]:
coordinates = mkCoordinate(task['LAT'],task['LONG'])
task.insert(value=coordinates,column='COORDINATES',loc=3)
task.head()

,USER,TWEET,CLASS,COORDINATES,TWEET_SEC,LAT,LONG
0,unatalclara,Que ironía que el tatuaje que me hice con mi e...,1.0,"[-99.1919955, 19.357102]","[28, 3394, 1, 8, 4176, 1, 10, 748, 22, 23, 554...",-99.191996,19.357102
1,alizamakeupmx,La ironía en el nombre del lugar #mames (@ Tra...,0.0,"[-99.259495, 19.3863707]","[59, 3394, 9, 8, 387, 27, 447, 2833, 3, 722, 1...",-99.259495,19.386371
2,etgarrido1976,"@Jashua98 @jrisco El Chairo, por definición, n...",0.0,"[-99.259495, 19.3863707]","[3, 3, 61, 16, 7, 1493, 5, 3394]",-99.259495,19.386371
3,enkianlatif,Jajaja que ironía el candidato del prian @Jose...,0.0,"[-99.1919955, 19.357102]","[323, 1, 3394, 8, 3100, 27, 3, 14, 57, 11]",-99.191996,19.357102
4,Raverp,#FactOfTheDay En todos los módulos de carga de...,1.0,"[-99.091915, 19.4170226]","[136, 87, 20, 12788, 2, 2822, 2, 7165, 6, 27, ...",-99.091915,19.417023


In [13]:
ultimate = task[['USER','TWEET','CLASS','COORDINATES']]
ultimate.head()

,USER,TWEET,CLASS,COORDINATES
0,unatalclara,Que ironía que el tatuaje que me hice con mi e...,1.0,"[-99.1919955, 19.357102]"
1,alizamakeupmx,La ironía en el nombre del lugar #mames (@ Tra...,0.0,"[-99.259495, 19.3863707]"
2,etgarrido1976,"@Jashua98 @jrisco El Chairo, por definición, n...",0.0,"[-99.259495, 19.3863707]"
3,enkianlatif,Jajaja que ironía el candidato del prian @Jose...,0.0,"[-99.1919955, 19.357102]"
4,Raverp,#FactOfTheDay En todos los módulos de carga de...,1.0,"[-99.091915, 19.4170226]"


In [14]:
alter = ultimate['COORDINATES'].apply(coor2json)
ultimate.insert(value=alter,column='JSON_COOR',loc=3)
ultimate.head()

,USER,TWEET,CLASS,JSON_COOR,COORDINATES
0,unatalclara,Que ironía que el tatuaje que me hice con mi e...,1.0,"{ ""type"": ""Point"", ""coordinates"": [-99.1919955...","[-99.1919955, 19.357102]"
1,alizamakeupmx,La ironía en el nombre del lugar #mames (@ Tra...,0.0,"{ ""type"": ""Point"", ""coordinates"": [-99.259495,...","[-99.259495, 19.3863707]"
2,etgarrido1976,"@Jashua98 @jrisco El Chairo, por definición, n...",0.0,"{ ""type"": ""Point"", ""coordinates"": [-99.259495,...","[-99.259495, 19.3863707]"
3,enkianlatif,Jajaja que ironía el candidato del prian @Jose...,0.0,"{ ""type"": ""Point"", ""coordinates"": [-99.1919955...","[-99.1919955, 19.357102]"
4,Raverp,#FactOfTheDay En todos los módulos de carga de...,1.0,"{ ""type"": ""Point"", ""coordinates"": [-99.091915,...","[-99.091915, 19.4170226]"


In [15]:
last = ultimate[['USER','TWEET','CLASS','JSON_COOR']]
last['TWEET'] = last['TWEET'].apply(utf2latam)
last.head()

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,USER,TWEET,CLASS,JSON_COOR
0,unatalclara,Que ironÃ­a que el tatuaje que me hice con mi ...,1.0,"{ ""type"": ""Point"", ""coordinates"": [-99.1919955..."
1,alizamakeupmx,La ironÃ­a en el nombre del lugar #mames (@ Tr...,0.0,"{ ""type"": ""Point"", ""coordinates"": [-99.259495,..."
2,etgarrido1976,"@Jashua98 @jrisco El Chairo, por definiciÃ³n, ...",0.0,"{ ""type"": ""Point"", ""coordinates"": [-99.259495,..."
3,enkianlatif,Jajaja que ironÃ­a el candidato del prian @Jos...,0.0,"{ ""type"": ""Point"", ""coordinates"": [-99.1919955..."
4,Raverp,#FactOfTheDay En todos los mÃ³dulos de carga d...,1.0,"{ ""type"": ""Point"", ""coordinates"": [-99.091915,..."


In [16]:
last.to_json('./supreme.json',orient='records')